In [32]:
####### module import 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
# Suppress Deprecation and Incorrect Usage Warnings 
import warnings
import scipy.io as sio 
import os
import scipy.signal as signal
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer , MinMaxScaler

warnings.filterwarnings('ignore')


In [33]:
def get_best_score(model):
    
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)
    
    return model.best_score_

In [34]:
path = '../ECoG_Handpose.mat'
data_set = sio.loadmat(path)['y']

columns = [f'CH{i}' for i in range(1, 68)]
df_ecog = pd.DataFrame(data_set.T, columns = columns)

In [35]:
df_ecog

,CH1,CH2,CH3,CH4,CH5,CH6,CH7,CH8,CH9,CH10,...,CH58,CH59,CH60,CH61,CH62,CH63,CH64,CH65,CH66,CH67
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.000833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.001667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.002500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.003333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507020,422.516667,-106697.578125,-86204.289062,-76535.585938,-103944.351562,-59508.328125,-39716.640625,-12350.508789,46311.046875,13856.023438,...,-65758.367188,-33892.039062,-4235.358887,51480.582031,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507021,422.517500,-106698.921875,-86203.906250,-76537.109375,-103945.375000,-59508.292969,-39717.875000,-12354.244141,46318.351562,13859.705078,...,-65760.468750,-33896.398438,-4240.333984,51477.101562,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507022,422.518333,-106701.960938,-86205.109375,-76535.445312,-103945.046875,-59511.664062,-39718.042969,-12355.633789,46322.660156,13859.116211,...,-65757.734375,-33894.804688,-4240.388672,51475.777344,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507023,422.519167,-106699.476562,-86202.367188,-76532.234375,-103945.843750,-59508.625000,-39713.183594,-12354.968750,46326.210938,13862.730469,...,-65757.828125,-33896.714844,-4241.463867,51473.503906,0.0,0.70303,0.178975,0.153902,0.161574,0.234733


In [23]:
df_ecog.describe()

,CH1,CH2,CH3,CH4,CH5,CH6,CH7,CH8,CH9,CH10,...,CH58,CH59,CH60,CH61,CH62,CH63,CH64,CH65,CH66,CH67
count,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,...,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000
mean,211.260000,-107458.068479,-83945.053488,-75942.404354,-103164.841146,-60166.487843,-39980.218778,-12617.111662,44837.251064,12756.745575,...,-65102.316006,-32529.935054,-2038.068605,51706.430555,0.868502,0.518840,0.247886,0.203706,0.399274,0.267168
std,121.971379,998.419347,1446.320544,990.284365,916.029018,1056.326673,1209.745580,1934.952619,1493.769177,2381.103458,...,899.869603,799.827436,1033.234762,729.529380,1.127756,0.172327,0.301131,0.209798,0.346444,0.234756
min,0.000000,-108750.093750,-86397.375000,-78346.015625,-104687.718750,-62050.808594,-42287.375000,-16689.199219,0.000000,0.000000,...,-97516.468750,-50292.171875,-6704.064453,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,105.630000,-108199.273438,-85126.500000,-76481.500000,-103784.914062,-61214.636719,-41402.292969,-14086.449219,43337.929688,11399.105469,...,-65684.984375,-33270.269531,-2742.752930,51407.484375,0.000000,0.329293,0.075524,0.093363,0.158796,0.085878
50%,211.260000,-107472.203125,-83737.554688,-75745.953125,-103064.968750,-59683.628906,-39513.355469,-11936.872070,45398.285156,13714.065430,...,-64922.500000,-32520.023438,-1627.766846,51896.367188,0.000000,0.573737,0.134398,0.134209,0.177778,0.173664
75%,316.890000,-106685.882812,-83194.710938,-75178.718750,-102528.617188,-59306.000000,-38937.097656,-11062.363281,46067.828125,14422.068359,...,-64333.898438,-31740.902344,-1153.763184,52262.371094,2.000000,0.662626,0.221395,0.180160,0.918519,0.433206
max,422.520000,56377.390625,56676.359375,56246.585938,56438.781250,56112.914062,56406.453125,56493.101562,56266.710938,56637.281250,...,0.000000,0.000000,0.000000,72322.281250,3.000000,0.791919,1.000000,0.824216,0.980093,1.000000


In [53]:
df_ecog[df_ecog['CH62'] == 0]

,CH1,CH2,CH3,CH4,CH5,CH6,CH7,CH8,CH9,CH10,...,CH58,CH59,CH60,CH61,CH62,CH63,CH64,CH65,CH66,CH67
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.000833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.001667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.002500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.003333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507020,422.516667,-106697.578125,-86204.289062,-76535.585938,-103944.351562,-59508.328125,-39716.640625,-12350.508789,46311.046875,13856.023438,...,-65758.367188,-33892.039062,-4235.358887,51480.582031,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507021,422.517500,-106698.921875,-86203.906250,-76537.109375,-103945.375000,-59508.292969,-39717.875000,-12354.244141,46318.351562,13859.705078,...,-65760.468750,-33896.398438,-4240.333984,51477.101562,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507022,422.518333,-106701.960938,-86205.109375,-76535.445312,-103945.046875,-59511.664062,-39718.042969,-12355.633789,46322.660156,13859.116211,...,-65757.734375,-33894.804688,-4240.388672,51475.777344,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507023,422.519167,-106699.476562,-86202.367188,-76532.234375,-103945.843750,-59508.625000,-39713.183594,-12354.968750,46326.210938,13862.730469,...,-65757.828125,-33896.714844,-4241.463867,51473.503906,0.0,0.70303,0.178975,0.153902,0.161574,0.234733


In [42]:
train_columns = [f'CH{i}' for i in range(2, 62)]
x_train, x_test, y_train, y_test = train_test_split(df_ecog[train_columns], df_ecog['CH62'], test_size = 0.15, shuffle=True, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, shuffle=True, random_state=1)

In [48]:
df_pred = pd.DataFrame(df_ecog['CH62'])
# df_pred['price'] = train['price']
df_pred.head()

,CH62
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [51]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

df_pred['LR_pred'] = pd.DataFrame(lr.predict(x_test))
print(lr.score(x_train,y_train))

0.5649581177344131


In [52]:
df_pred

,CH62,LR_pred
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
507020,0.0,NaN
507021,0.0,NaN
507022,0.0,NaN
507023,0.0,NaN
